In [15]:
!ray stop --force

(raylet) Raylet is terminated. Termination is unexpected. Possible reasons include: (1) SIGKILL by the user or system OOM killer, (2) Invalid memory access from Raylet causing SIGSEGV or SIGBUS, (3) Other termination signals. Last 20 lines of the Raylet logs:
    [state-dump] 	ray::rpc::InternalPubSubGcsService.grpc_client.GcsSubscriberPoll - 4 total (1 active), Execution time: mean = 1.610 s, total = 6.439 s, Queueing time: mean = 0.000 ns, max = -1.000 ns, min = 9223372036.855 s, total = 0.000 ns
    [state-dump] 	ray::rpc::InternalPubSubGcsService.grpc_client.GcsSubscriberPoll.OnReplyReceived - 3 total (0 active), Execution time: mean = 94.667 us, total = 284.000 us, Queueing time: mean = 75.000 us, max = 191.000 us, min = 9.000 us, total = 225.000 us
    [state-dump] 	ray::rpc::InternalPubSubGcsService.grpc_client.GcsSubscriberCommandBatch - 2 total (0 active), Execution time: mean = 242.500 us, total = 485.000 us, Queueing time: mean = 0.000 ns, max = -1.000 ns, min = 9223372036.8

[2025-05-09 20:54:25,517 E 9102 259458] core_worker.cc:941: :info_message: Attempting to recover 102 lost objects by resubmitting their tasks or setting a new primary location from existing copies. To disable object reconstruction, set @ray.remote(max_retries=0).


Stopped only 14 out of 20 Ray processes within the grace period 16 seconds. Set `-v` to see more details. Remaining processes [psutil.Process(pid=9464, name='raylet', status='zombie', started='20:47:05'), psutil.Process(pid=9457, name='python3.12', status='zombie', started='20:47:04'), psutil.Process(pid=9459, name='python3.12', status='zombie', started='20:47:04'), psutil.Process(pid=9460, name='python3.12', status='zombie', started='20:47:04'), psutil.Process(pid=9465, name='python3.12', status='zombie', started='20:47:05'), psutil.Process(pid=9458, name='gcs_server', status='zombie', started='20:47:04')] will be forcefully terminated.
You can also use `--force` to forcefully terminate processes or set higher `--grace-period` to wait longer time for proper termination.


In [1]:
import ray
ray.init()

2025-05-09 20:58:07,897	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-09 20:58:08,782	INFO worker.py:1888 -- Started a local Ray instance.


Python version:,3.12.4
Ray version:,2.46.0


In [2]:
import ray
import ray.data
import requests


# Step 1: Fetch disaster data from ReliefWeb API
def fetch_crisis_data():
    url = "https://api.reliefweb.int/v1/disasters?appname=crisis-app&limit=100"
    response = requests.get(url)
    data = response.json()["data"]

    def extract_info(item):
        fields = item["fields"]
        return {
            "name": fields.get("name"),
            "type": fields.get("type")[0]["name"] if fields.get("type") else None,
            "country": fields.get("country")[0]["name"] if fields.get("country") else None,
            "date": fields.get("date", {}).get("created"),
            "description": fields.get("description", "") or fields.get("name", "")
        }

    return [extract_info(d) for d in data]

# Step 2: Load data into Ray Dataset
records = fetch_crisis_data()
ds = ray.data.from_items(records)

# Optional: Filter for floods only
ds_filtered = ds.filter(lambda row: row["type"] == "Flood")

# Step 3: Define summarization using transformers (Flan-T5)
def summarize_batch(df):
    from transformers import pipeline
    summarizer = pipeline("summarization", model="google/flan-t5-base")
    df["summary"] = df["description"].apply(
        lambda text: summarizer(text, max_length=60, min_length=10, do_sample=False)[0]["summary_text"]
    )
    return df

# Step 4: Run summarization with map_batches (safe for Windows)
summarized_ds = ds_filtered.map_batches(
    summarize_batch,
    batch_format="pandas",
    batch_size=4
)

# Step 5: Show results
summarized_ds.show(5)

# Optional: Save to file
summarized_ds.write_parquet("summarized_crisis.parquet")


2025-05-09 20:58:12,466	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/anaconda3/lib/python3.12/site-packages/ray/data/dataset.py:1413: UserWarning: Use 'expr' instead of 'fn' when possible for performant filters.
  warnings.warn(
2025-05-09 20:58:12,989	INFO dataset.py:3027 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-05-09 20:58:12,992	INFO logging.py:290 -- Registered dataset logger for dataset dataset_3_0
2025-05-09 20:58:13,002	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_3_0. Full logs are in /tmp/ray/session_2025-05-09_20-58-07_921291_9784/logs/ray-data
2025-05-09 20:58:13,002	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_3_0: InputDataBuffer[Input] -> TaskPoolMapOperator[Filter(<lambda>)] -> TaskPoolMapOperator[MapBatches(summariz

Running 0: 0.00 row [00:00, ? row/s]

- Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- MapBatches(summarize_batch) 2: 0.00 row [00:00, ? row/s]

- limit=5 3: 0.00 row [00:00, ? row/s]

2025-05-09 20:58:13,977	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_3_0 execution finished in 0.97 seconds
2025-05-09 20:58:14,017	INFO logging.py:290 -- Registered dataset logger for dataset dataset_5_0
2025-05-09 20:58:14,019	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_5_0. Full logs are in /tmp/ray/session_2025-05-09_20-58-07_921291_9784/logs/ray-data
2025-05-09 20:58:14,020	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_5_0: InputDataBuffer[Input] -> TaskPoolMapOperator[Filter(<lambda>)] -> TaskPoolMapOperator[MapBatches(summarize_batch)->Write]


Running 0: 0.00 row [00:00, ? row/s]

- Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- MapBatches(summarize_batch)->Write 2: 0.00 row [00:00, ? row/s]

2025-05-09 20:58:14,146	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_5_0 execution finished in 0.13 seconds
2025-05-09 20:58:14,152	INFO dataset.py:4537 -- Data sink Parquet finished. 0 rows and 0.0B data written.


In [14]:
def summarize_batch(df):
    from transformers import pipeline
    summarizer = pipeline("summarization", model="google/flan-t5-base")

    # Generate summaries
    df["summary"] = df["description"].apply(lambda text: summarizer(text, max_length=60, min_length=10, do_sample=False)[0]["summary_text"])
    return df

# Run summarization in batches using pandas format
summarized_ds = ds_filtered.map_batches(
    summarize_batch,
    batch_format="pandas",  # use pandas to avoid serialization issues
    batch_size=2
)

# Show the result
summarized_ds.show()

2025-05-09 20:54:07,267	INFO logging.py:290 -- Registered dataset logger for dataset dataset_11_0
2025-05-09 20:54:07,269	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_11_0. Full logs are in /tmp/ray/session_2025-05-09_20-47-04_703001_9102/logs/ray-data
2025-05-09 20:54:07,270	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_11_0: InputDataBuffer[Input] -> TaskPoolMapOperator[Filter(<lambda>)] -> TaskPoolMapOperator[MapBatches(summarize_batch)] -> LimitOperator[limit=20]


Running 0: 0.00 row [00:00, ? row/s]

- Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- MapBatches(summarize_batch) 2: 0.00 row [00:00, ? row/s]

- limit=20 3: 0.00 row [00:00, ? row/s]

2025-05-09 20:54:07,390	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_11_0 execution finished in 0.12 seconds
